In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report, confusion_matrix

Read YOLO data & process

In [2]:
# DATA PATH
path = 'final_dataset_withnon'
train_img_path = os.path.join(path, 'images', 'train')
train_lbl_path = os.path.join(path, 'labels', 'train')

valid_img_path = os.path.join(path, 'images', 'val')
valid_lbl_path = os.path.join(path, 'labels', 'val')

test_img_path = os.path.join(path, 'images', 'test')
test_lbl_path = os.path.join(path, 'labels', 'test')

IMG_SIZE = 128
NUM_CLASSES = 4

# path to save processed images
processed_path = os.path.join(path, 'processed')
os.makedirs(processed_path, exist_ok=True)

In [3]:
# process a dataset by extracting craters from images based on YOLO-format labels and saving them in class-specific directories. 
def process_dataset(img_dir, lbl_dir, output_dir):
    for img_file in os.listdir(img_dir):
        if not img_file.endswith('.jpg'):
            continue # skip non-image files
            
        # Get the corresponding label file
        base_name = os.path.splitext(img_file)[0]
        lbl_file = os.path.join(lbl_dir, f"{base_name}.txt")
        
        # Process single image
        img = Image.open(os.path.join(img_dir, img_file))
        img_w, img_h = img.size
        
        with open(lbl_file, 'r') as f:
            for idx, line in enumerate(f.readlines()):
                class_id, xc, yc, w, h = map(float, line.strip().split())
                # Ensure image bounds
                x1 = int((xc - w/2) * img_w)
                y1 = int((yc - h/2) * img_h)
                x2 = int((xc + w/2) * img_w)
                y2 = int((yc + h/2) * img_h)
                                
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(img_w, x2), min(img_h, y2)
                
                # Crop the crater and resize it
                crater = img.crop((x1, y1, x2, y2))
                crater = crater.resize((IMG_SIZE, IMG_SIZE), Image.Resampling.LANCZOS)
                # save
                class_dir = os.path.join(output_dir, str(int(class_id)))
                os.makedirs(class_dir, exist_ok=True)
                crater.save(os.path.join(class_dir, f"{base_name}_{idx}.jpg"))

# process all dataset
process_dataset(train_img_path, train_lbl_path, os.path.join(processed_path, 'train'))
process_dataset(valid_img_path, valid_lbl_path, os.path.join(processed_path, 'val'))
process_dataset(test_img_path, test_lbl_path, os.path.join(processed_path, 'test'))

In [4]:
# Define data enhancement
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# Loading data from category folder
train_generator = train_datagen.flow_from_directory(
    os.path.join(processed_path, 'train'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical'
)

val_generator = train_datagen.flow_from_directory(
    os.path.join(processed_path, 'val'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical'
)

# test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    os.path.join(processed_path, 'test'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical', 
    shuffle=False
)

Found 1880 images belonging to 4 classes.
Found 413 images belonging to 4 classes.
Found 872 images belonging to 4 classes.


# Resnet 50

In [5]:
from sklearn.metrics import f1_score as sk_f1

def multi_class_f1(y_true, y_pred):
    def f1_np(y_true_np, y_pred_np):
        y_true_np = np.argmax(y_true_np, axis=1)
        y_pred_np = np.argmax(y_pred_np, axis=1)
        f1 = sk_f1(y_true_np, y_pred_np, average='weighted')
        return np.array(f1, dtype=np.float32)

    f1_score = tf.py_function(func=f1_np, inp=[y_true, y_pred], Tout=tf.float32)
    f1_score.set_shape([])
    return f1_score


In [16]:
ResNet50_model = ResNet50(
    include_top=False,
    weights=None,
    input_shape=(128, 128, 3)
)

# for layer in ResNet50_model.layers[:-10]:
#     layer.trainable = False
# 
# for layer in ResNet50_model.layers[-10:]:
#     layer.trainable = True

# defined output layer
x = layers.GlobalAveragePooling2D()(ResNet50_model.output)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation='relu', kernel_regularizer=l2(1e-4))(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(4, activation='softmax')(x)

model = models.Model(inputs=ResNet50_model.input, outputs=x)

model.compile(
    optimizer = Adam(learning_rate=0.001), 
    loss='categorical_crossentropy',
    metrics=[multi_class_f1]
)

checkpoint = ModelCheckpoint(
    'best_model_resnet.keras', 
    monitor='val_multi_class_f1', 
    save_best_only=True, 
    mode='max'
)
# early_stop = EarlyStopping(
#     monitor='val_multi_class_f1',
#     patience=7,
#     restore_best_weights=True,
#     mode='max',
#     verbose=1
# )

reduce_lr = ReduceLROnPlateau(monitor='val_multi_class_f1', factor=0.5, patience=5, mode='max',verbose=0)

history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[checkpoint, reduce_lr]
)


Epoch 1/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 71s 901ms/step - loss: 1.7963 - multi_class_f1: 0.7448 - val_loss: 0.6225 - val_multi_class_f1: 0.8484 - learning_rate: 0.0010
Epoch 2/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 50s 837ms/step - loss: 1.0167 - multi_class_f1: 0.8190 - val_loss: 0.7972 - val_multi_class_f1: 0.8471 - learning_rate: 0.0010
Epoch 3/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 51s 859ms/step - loss: 0.7347 - multi_class_f1: 0.8146 - val_loss: 11.0686 - val_multi_class_f1: 0.8481 - learning_rate: 0.0010
Epoch 4/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - loss: 1.3203 - multi_class_f1: 0.7886 - val_loss: 2400.0945 - val_multi_class_f1: 0.8464 - learning_rate: 0.0010
Epoch 5/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - loss: 1.1023 - multi_class_f1: 0.7813 - val_loss: 6.0972 - val_multi_class_f1: 0.8492 - learning_rate: 0.0010
Epoch 6/30
59/59 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - loss: 0.8749 - multi_class_f1: 0.7725 - val_loss: 0.5815 - val_multi_class_f1: 0.8304 - learning_rate: 0.0010
Epoch 7/30
59/5

In [17]:
best_model_resnet = load_model(
    'best_model_resnet.keras',
    custom_objects={'multi_class_f1': multi_class_f1}
)
test_loss, test_f1 = best_model_resnet.evaluate(test_generator)
print(f'\n✅ Test Macro F1-score: {test_f1:.4f}')
print(f'✅ Test loss: {test_loss:.4f}')


28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 188ms/step - loss: 0.2796 - multi_class_f1: 0.9353

✅ Test Macro F1-score: 0.9435
✅ Test loss: 0.2699


In [18]:
y_pred_probs = best_model_resnet.predict(test_generator)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes 

class_names = list(test_generator.class_indices.keys())

report = classification_report(
    y_true, y_pred_classes,
    target_names=class_names,
    output_dict=True,
    zero_division=1
)

print("\n📊 Per-Class F1-scores:")
for class_name in class_names:
    f1 = report[class_name]['f1-score']
    print(f"  {class_name}: {f1:.4f}")


print("\n📃 Full classification report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names, zero_division=1))


conf_mat = confusion_matrix(y_true, y_pred_classes)
print("\n🧮 Confusion Matrix:")
print(conf_mat)

28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 255ms/step

📊 Per-Class F1-scores:
  0: 0.4384
  1: 0.9742
  2: 0.4925
  3: 0.8000

📃 Full classification report:
              precision    recall  f1-score   support

           0       0.38      0.52      0.44        31
           1       0.99      0.96      0.97       746
           2       0.48      0.51      0.49        65
           3       0.74      0.87      0.80        30

    accuracy                           0.91       872
   macro avg       0.65      0.71      0.68       872
weighted avg       0.92      0.91      0.91       872


🧮 Confusion Matrix:
[[ 16   0  11   4]
 [  2 717  25   2]
 [ 23   6  33   3]
 [  1   3   0  26]]


### 30 times Exp

In [6]:
IMG_SIZE = 128
NUM_CLASSES = 4
NUM_RUNS = 30
class_names = list(test_generator.class_indices.keys())

metrics = {
    'classes': {cls: {'precision': [], 'recall': [], 'f1': []} for cls in class_names},
    'macro_avg': {'precision': [], 'recall': [], 'f1': []},
    'weighted_avg': {'precision': [], 'recall': [], 'f1': []},
    'accuracy': []
}


def create_resnet_model():
    ResNet50_model = ResNet50(
    include_top=False,
    weights=None,
    input_shape=(128, 128, 3)
    )
    
    # for layer in ResNet50_model.layers[:-10]:
    #     layer.trainable = False
    # 
    # for layer in ResNet50_model.layers[-10:]:
    #     layer.trainable = True
    
    
    # defined output layer
    x = layers.GlobalAveragePooling2D()(ResNet50_model.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(4, activation='softmax')(x)
    
    model = models.Model(inputs=ResNet50_model.input, outputs=x)
    
    model.compile(
        optimizer = Adam(learning_rate=0.001), 
        loss='categorical_crossentropy',
        metrics=[multi_class_f1]
    )
    return model

TEMP_MODEL_PATH = 'temp_best_model_resnet.keras'


for run in range(NUM_RUNS):
    print(f"\n=========== Training Run {run + 1}/{NUM_RUNS} ===========")
    
    if os.path.exists(TEMP_MODEL_PATH):
        os.remove(TEMP_MODEL_PATH)

    tf.keras.backend.clear_session()
    np.random.seed(run)
    tf.random.set_seed(run)

    model = create_resnet_model()

    checkpoint = ModelCheckpoint('temp_best_model_resnet.keras', monitor='val_multi_class_f1', save_best_only=True, mode='max', verbose=0)
    
    reduce_lr = ReduceLROnPlateau(monitor='val_multi_class_f1', factor=0.5, patience=5, mode='max',verbose=0)
    
    model.fit(
        train_generator,
        epochs=30,
        validation_data=val_generator,
        callbacks=[checkpoint, reduce_lr],
        verbose=0
    )

    best_model = load_model('temp_best_model_resnet.keras', custom_objects={'multi_class_f1': multi_class_f1})

    y_pred = best_model.predict(test_generator, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = test_generator.classes

    report = classification_report(y_true, y_pred_classes,
                                   target_names=class_names,
                                   output_dict=True,
                                   zero_division=1)

    for cls in class_names:
        metrics['classes'][cls]['precision'].append(report[cls]['precision'])
        metrics['classes'][cls]['recall'].append(report[cls]['recall'])
        metrics['classes'][cls]['f1'].append(report[cls]['f1-score'])

    metrics['macro_avg']['precision'].append(report['macro avg']['precision'])
    metrics['macro_avg']['recall'].append(report['macro avg']['recall'])
    metrics['macro_avg']['f1'].append(report['macro avg']['f1-score'])

    metrics['weighted_avg']['precision'].append(report['weighted avg']['precision'])
    metrics['weighted_avg']['recall'].append(report['weighted avg']['recall'])
    metrics['weighted_avg']['f1'].append(report['weighted avg']['f1-score'])

    metrics['accuracy'].append(report['accuracy'])





=========== Training Run 1/30 ===========



C:\Users\10302\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
C:\Users\10302\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
C:\Users\10302\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().


=========== Training Run 2/30 ===========

=========== Training Run 3/30 ===========

=========== Training Run 4/30 ===========

=========== Training Run 5/30 ===========

=========== Training Run 6/30 ===========

=========== Training Run 7/30 ===========

=========== Training Run 8/30 ===========

=========== Training Run 9/30 ===========

=========== Training Run 10/30 ===========

=========== Training Run 11/30 ===========

=========== Training Run 12/30 ===========

=========== Training Run 13/30 ===========

=========== Training Run 14/30 ===========

=========== Training Run 15/30 ===========

=========== Training Run 16/30 ===========

=========== Training Run 17/30 ===========

=========== Training Run 18/30 ===========

=========== Training Run 19/30 ===========

=========== Training Run 20/30 ===========

=========== Training Run 21/30 ===========

=========== Training Run 22/30 ===========

=========== Training Run 23/30 ===========

=========== Training Run 24/30 ========

In [7]:
print("\n\n=== Classification Report with Mean±Std ===")

class_col_width = max(len(str(cls)) for cls in class_names) + 2
metric_width = 14
support_width = 8

print(f"\n{'Class':<{class_col_width}} {'Precision':<{metric_width}} {'Recall':<{metric_width}} {'F1-score':<{metric_width}} {'Support':<{support_width}}")

for idx, cls in enumerate(class_names):
    prec_mean = np.mean(metrics['classes'][cls]['precision'])
    prec_std = np.std(metrics['classes'][cls]['precision'])
    rec_mean = np.mean(metrics['classes'][cls]['recall'])
    rec_std = np.std(metrics['classes'][cls]['recall'])
    f1_mean = np.mean(metrics['classes'][cls]['f1'])
    f1_std = np.std(metrics['classes'][cls]['f1'])
    support = test_generator.classes.tolist().count(idx)

    print(f"{cls:<{class_col_width}} "
          f"{prec_mean:.2f}±{prec_std:.2f}  "
          f"{rec_mean:.2f}±{rec_std:.2f}  "
          f"{f1_mean:.2f}±{f1_std:.2f}  "
          f"{support:<{support_width}}")

def print_avg_row(name, metric_dict):
    prec = f"{np.mean(metric_dict['precision']):.2f}±{np.std(metric_dict['precision']):.2f}"
    rec = f"{np.mean(metric_dict['recall']):.2f}±{np.std(metric_dict['recall']):.2f}"
    f1 = f"{np.mean(metric_dict['f1']):.2f}±{np.std(metric_dict['f1']):.2f}"
    print(f"{name:<{class_col_width}} {prec:<{metric_width}} {rec:<{metric_width}} {f1:<{metric_width}}")

# accuracy
acc_mean = np.mean(metrics['accuracy'])
acc_std = np.std(metrics['accuracy'])
total_samples = len(test_generator.classes)

print(f"\n{'accuracy':<{class_col_width}} {'':<{metric_width}} {'':<{metric_width}} "
      f"{acc_mean:.2f}±{acc_std:.2f}  {total_samples:<{support_width}}")

print_avg_row('macro avg', metrics['macro_avg'])
print_avg_row('weighted avg', metrics['weighted_avg'])




=== Classification Report with Mean±Std ===

Class Precision      Recall         F1-score       Support 
0   0.38±0.17  0.26±0.20  0.27±0.16  31      
1   0.97±0.02  0.96±0.05  0.96±0.03  746     
2   0.52±0.12  0.62±0.18  0.54±0.12  65      
3   0.71±0.17  0.65±0.18  0.65±0.14  30      

accuracy                               0.90±0.05  872     
macro avg 0.65±0.09      0.62±0.07      0.61±0.07     
weighted avg 0.91±0.02      0.90±0.05      0.90±0.03     
